Procedo a modelar los datasets que voy a utilizar para las APIs, de manera que consuman menos memoria y espacio, utilizando <br>
solo el contenido y datos necesarios

In [1]:
import pandas as pd
from textblob import TextBlob
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df_steam = pd.read_csv('../FinalData/data_steam_t.csv', encoding='UTF_8')
df_reviews = pd.read_csv('../FinalData/data_reviews_t.csv', encoding='UTF_8')
df_items = pd.read_csv('../FinalData/data_items_t.csv', encoding='UTF_8')

In [3]:
df_steam.head()

,publisher,genres,app_name,tags,price,early_access,item_id,developer,release_year
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,0.0,761140,Kotoshiro,2018
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",0.00,0.0,643980,Secret Level SRL,2018
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...",0.00,0.0,670290,Poolians.com,2017
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,"['Action', 'Adventure', 'Casual']",0.99,0.0,767400,彼岸领域,2017
4,NaN,"['Action', 'Indie', 'Casual', 'Sports']",Log Challenge,"['Action', 'Indie', 'Casual', 'Sports']",2.99,0.0,773570,NaN,No data


In [4]:
df_reviews.head()

,posted,item_id,recommend,review,user_id
0,2011,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,2011,22200,True,It's unique and worth a playthrough.,76561197970982479
2,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,2014,251610,True,I know what you think when you see this title ...,js41637
4,2013,227300,True,For a simple (it's actually not all that simpl...,js41637


In [5]:
df_items.head()

,user_id,steam_id,item_id,item_name,playtime_forever
0,76561197970982479,76561197970982479,10,Counter-Strike,6
1,76561197970982479,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,76561197970982479,30,Day of Defeat,7
3,76561197970982479,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,76561197970982479,50,Half-Life: Opposing Force,0


In [6]:
### Procedo a hacer un 'sentiment_analysis' para df_reviews 
#   aplicando análisis de sentimiento con NLP con la siguiente escala: 
#   debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo


df_reviews['sentiment_analysis'] = df_reviews['review'].apply(lambda x: TextBlob(str(x)).sentiment.polarity if pd.notna(x) and x != 'No data' else None)

# Define a function to convert the sentiment analysis to the scale (0, 1, 2)
def map_sentiment(score):
    if score is None:
        return 1  # If the review is absent or 'No data', take the value of 1
    elif score < 0:
        return 0  # bad
    elif score == 0:
        return 1  # neutral
    else:
        return 2  # positive

# Apply the mapping function to the 'sentiment_analysis' column
df_reviews['sentiment_analysis'] = df_reviews['sentiment_analysis'].apply(map_sentiment)

In [8]:
### Visualizo el dataframe con la nueva columna
df_reviews

,posted,item_id,recommend,review,user_id,sentiment_analysis
0,2011,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,2011,22200,True,It's unique and worth a playthrough.,76561197970982479,2
2,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,2014,251610,True,I know what you think when you see this title ...,js41637,2
4,2013,227300,True,For a simple (it's actually not all that simpl...,js41637,0
...,...,...,...,...,...,...
58426,No data,70,True,a must have classic from steam definitely wort...,76561198312638244,2
58427,No data,362890,True,this game is a perfect remake of the original ...,76561198312638244,2
58428,No data,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,2
58429,No data,730,True,:D,LydiaMorley,2


In [9]:
df_reviews['sentiment_analysis'].unique()

array([2, 0, 1], dtype=int64)

### Modelo dframe que voy a utilizar para la funcion PlaytimeGenre

In [10]:
# Vamos a trabajar con la columna 'genres' en una copia de df_steam

df_PTG = df_steam.copy()
# Aquí, estoy usando apply() con una función lambda para unir los elementos de una lista
# si es que la columna 'genres' contiene listas. Si lo hace, los elementos se unen con comas y espacios.
# Si no es una lista (es decir, si no es del tipo list), mantengo el valor original sin modificar.
df_PTG['genres'] = df_PTG['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Ahora, estoy aplicando otra función lambda a la columna 'genres'.
# Esta vez, estoy verificando si los valores son cadenas de texto (str).
# Si es una cadena, elimino los corchetes '[' y ']' alrededor del texto con el método strip('[]').
# Esto es útil para limpiar el formato de los datos si hay corchetes alrededor de las cadenas.
# Si no es una cadena, simplemente mantengo el valor original sin cambios.
df_PTG['genres'] = df_PTG['genres'].apply(lambda x: x.strip('[]') if isinstance(x, str) else x)

df_PTG = df_PTG.assign(genres=df_PTG['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else [x])).explode('genres').reset_index(drop=True)
df_PTG['genres']=df_PTG['genres'].str.replace("'",'')
df_PTG['genres']=df_PTG['genres'].str.replace("\"",'')
df_PTG['genres']=df_PTG['genres'].str.replace(" ",'')
df_PTG

,publisher,genres,app_name,tags,price,early_access,item_id,developer,release_year
0,Kotoshiro,Action,Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,0.0,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,0.0,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,0.0,761140,Kotoshiro,2018
3,Kotoshiro,Simulation,Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,0.0,761140,Kotoshiro,2018
4,Kotoshiro,Strategy,Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.99,0.0,761140,Kotoshiro,2018
...,...,...,...,...,...,...,...,...,...
85514,NaN,Adventure,Maze Run VR,"['Early Access', 'Adventure', 'Indie', 'Action...",4.99,1.0,681550,NaN,No data
85515,NaN,Indie,Maze Run VR,"['Early Access', 'Adventure', 'Indie', 'Action...",4.99,1.0,681550,NaN,No data
85516,NaN,Action,Maze Run VR,"['Early Access', 'Adventure', 'Indie', 'Action...",4.99,1.0,681550,NaN,No data
85517,NaN,Simulation,Maze Run VR,"['Early Access', 'Adventure', 'Indie', 'Action...",4.99,1.0,681550,NaN,No data


In [11]:
### def PlayTimeGenre

### Necesito los datos de item_id, genres y release de la lista de steamGames, para combinarlos con los datos de 
### playtime_forever e item_id de df_items

df_playtime_genre = df_PTG[['item_id','genres','release_year']].merge(df_items[['playtime_forever', 'item_id']], on='item_id')
df_playtime_genre = df_playtime_genre.groupby(['genres', 'release_year'])['playtime_forever'].sum().reset_index()

In [12]:
df_playtime_genre

,genres,release_year,playtime_forever
0,1980s,2015,23930
1,1980s,No data,3951
2,1990s,2011,172945
3,1990s,2012,57473
4,2D,2012,167402
...,...,...,...
789,WorldWarII,No data,43035305
790,Zombies,2015,2728
791,Zombies,No data,78741
792,e-sports,2014,448600


In [13]:
df_playtime_genre.to_csv('../FeatureEngData/df_playtime_genre.csv', index=False)

In [14]:
### UserForGenre
### Necesito los datos de item_id, genres, release_year de SteamGames para combinarlos con 'playtime_forever','user_id','item_id'
### de df_items


df_UFG = df_PTG[['item_id', 'genres', 'release_year']].merge(df_items[['playtime_forever','user_id','item_id']],on='item_id')
df_UFG

,item_id,genres,release_year,playtime_forever,user_id
0,282010,Action,1997,5,UTNerd24
1,282010,Action,1997,0,I_DID_911_JUST_SAYING
2,282010,Action,1997,0,76561197962104795
3,282010,Action,1997,0,r3ap3r78
4,282010,Action,1997,13,saint556
...,...,...,...,...,...
11341506,80,Action,2004,0,76561198273508956
11341507,80,Action,2004,0,76561198282090798
11341508,80,Action,2004,0,943525
11341509,80,Action,2004,9,76561198283312749


In [15]:
df_UFG['playtime_forever'].sort_values(ascending=False)

55300      642773
98149      642773
1610129    635295
1612463    635295
1614797    635295
            ...  
4368772         0
4368771         0
4368766         0
4368764         0
5670755         0
Name: playtime_forever, Length: 11341511, dtype: int64

In [16]:
df_UFG['horas'] = (df_UFG['playtime_forever'] / 60).round(2)


In [17]:
df_UFG = df_UFG.groupby(['genres', 'user_id', 'release_year'])['horas'].sum().reset_index()
df_UFG


,genres,user_id,release_year,horas
0,1980s,-Mad-,2015,0.00
1,1980s,00690069006900,2015,0.00
2,1980s,007james_bond,2015,0.50
3,1980s,08alross,2015,0.00
4,1980s,091263,2015,0.00
...,...,...,...,...
4592903,e-sports,zuilde,2014,0.12
4592904,e-sports,zwanzigdrei,2014,8.53
4592905,e-sports,zyxwvutsrqponm,2014,0.00
4592906,e-sports,zzeee,2014,0.02


In [18]:
type(df_UFG['release_year'][3])

str

In [31]:
### Dado que si exporto el archivo en csv sobrepasa los 150MB, voy a utilizar parquet para reducir consumo de almacenamiento
import pyarrow
df_UFG.to_parquet('../FeatureEngData/df_user_for_genre.parquet',index=False)

In [20]:
### UserRecommend
### Necesito las recomendaciones, sentimentanalysis, ids y año, de steam games y reviews

df_UR = df_steam[['item_id','app_name']].merge(df_reviews[['item_id', 'posted', 'sentiment_analysis', 'recommend']], on='item_id')
df_UR

,item_id,app_name,posted,sentiment_analysis,recommend
0,282010,Carmageddon Max Pack,No data,1,True
1,70,Half-Life,2015,0,True
2,70,Half-Life,2011,0,True
3,70,Half-Life,2014,0,True
4,70,Half-Life,2013,2,True
...,...,...,...,...,...
53179,80,Counter-Strike: Condition Zero,2014,1,False
53180,80,Counter-Strike: Condition Zero,2013,2,True
53181,80,Counter-Strike: Condition Zero,2015,2,True
53182,80,Counter-Strike: Condition Zero,2014,1,True


In [21]:
df_UR.to_csv('../FeatureEngData/df_users_recommend.csv',index=False)

### Utilizo dfUR tambien para usersNOTrecommend

In [22]:
### SentimentAnalysis
### Necesito el itemid y año de dfSteam
### junto con item id y sentimentanalysis de reviews

df_PTG = df_PTG[df_PTG['release_year'] != 'no data']

df_SA = df_PTG[['item_id', 'release_year']].merge(df_reviews[['item_id', 'sentiment_analysis']], on='item_id')

In [23]:
df_SA.to_csv('../FeatureEngData/df_sentimennt_analysis.csv',index=False)

In [24]:
# Selecciono las columnas 'genres', 'tags', 'app_name', 'item_id' del DataFrame df_steam para el modelo
df_user_recomendation = df_steam[['genres', 'tags', 'app_name', 'item_id']].copy()

# Convierto la columna 'tags' en una cadena separada por comas si es una lista
df_user_recomendation['tags'] = df_user_recomendation['tags'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Elimino los caracteres '[' y ']' de la columna 'tags' si es una cadena
df_user_recomendation['tags'] = df_user_recomendation['tags'].apply(lambda x: x.strip('[]') if isinstance(x, str) else x)

# Elimino las comillas simples de la columna 'tags' y 'genres' si están presentes
df_user_recomendation['tags'] = df_user_recomendation['tags'].str.replace("'", '')
df_user_recomendation['genres'] = df_user_recomendation['genres'].str.replace("'", '')

# Selecciono una muestra aleatoria del 30% del DataFrame con una semilla aleatoria de 321
df_user_recomendation = df_user_recomendation.sample(frac=0.3, random_state=321)

# Creo una nueva columna 'combinacion' concatenando las columnas 'genres', 'tags' y 'app_name'
df_user_recomendation['combinacion'] = df_user_recomendation['genres'].fillna('') + ' ' + df_user_recomendation['tags'].fillna('') + ' ' + df_user_recomendation['app_name'].fillna('')

# Reinicio los índices del DataFrame después de seleccionar una muestra aleatoria
df_user_recomendation = df_user_recomendation.reset_index(drop=True)


In [25]:
df_user_recomendation.to_csv('../FeatureEngData/df_recomendacion_juego.csv', index= False)

In [26]:
### Preprocesamiento de datos

# Creo una matriz TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_user_recomendation['combinacion'])

# Calculo la similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
def recomendacion_juego(producto_id, cosine_sim=cosine_sim, df=df_user_recomendation):
    # Verifico si el ID del producto está presente en el DataFrame
    if producto_id not in df['item_id'].values:
        return "No se encontraron datos para este ID de producto."
    
    idx = df[df['item_id'] == producto_id].index[0]  # Obtener el índice del juego dado su ID
    
    # Calcular la similitud entre el juego dado y los demás juegos
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5 juegos similares
    
    # Obtener los índices de los juegos similares
    game_indices = [i[0] for i in sim_scores]
    
    # Devolver los nombres de los juegos recomendados
    return df['app_name'].iloc[game_indices]



In [27]:
recomendacion_juego(734700)

2921        Lost Route
9218            M.I.A.
6849    Lost Shipwreck
8557         Echo Nine
8481           C O S M
Name: app_name, dtype: object

In [28]:
df_user_recomendation_2 = df_user_recomendation.copy().merge(df_steam)